A mixture of $N =10$ random Gaussians centered in $[-1,1]^n$ 
$$
f_{\mu,\sigma}(x, y) = \sum_{i=1}^{N} \frac{1}{\sigma_i\sqrt{2\pi}}\exp\left(-\frac{(x-\mu_i)^2}{2\sigma_i^2}\right)
$$
 is defined over the square $[-1, 1]^2$.

In [2]:
using Pkg
Pkg.activate("../../.")
using Globtim
using DynamicPolynomials, DataFrames
using HomotopyContinuation, ProgressLogging

  Activating project at `~/Globtim.jl`


In [3]:
# Constants and Parameters
const n, a, b = 2, 1, 1
const scale_factor = a / b       # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = 0.5, 1 / 10  # Sampling parameters
const tol_l2 = 1.5e-3             # Define the tolerance for the L2-norm

N = 12
params = init_gaussian_params(n, N, 0.2)
# Create a closure that captures params
rand_gaussian_closure = (x) -> rand_gaussian(x, params)
f = rand_gaussian_closure; # Function to be optimized

In [4]:
d = 4 # Initial Degree 
center = [0.0, 0.0]
TR = test_input(n, center, (alpha, delta), tol_l2, (0.0, 0.0), 1.0, 1.0, f)
pol_cheb = Constructor(TR, d, GN=10, basis=:chebyshev)
pol_lege = Constructor(TR, d, GN=10, basis=:legendre);

current L2-norm: 0.030777449424258453
Number of samples: 10
current L2-norm: 0.033137300294305935
Number of samples: 10


In [11]:
@polyvar(x[1:n]) # Define polynomial ring 
pol = main_nd(x, n, d, pol_cheb.coeffs, basis=:chebyshev) 
grad = differentiate.(pol, x)
sys = HomotopyContinuation.System(grad)
println("The system is of degree: ", d - 1)
Real_sol_lstsq = HomotopyContinuation.solve(sys)
real_pts_cheb = HomotopyContinuation.real_solutions(Real_sol_lstsq; only_real=true, multiple_results=false);

The system is of degree: 3


In [12]:
pol = main_nd(x, n, d, pol_lege.coeffs, basis=:legendre)
grad = differentiate.(pol, x)
sys = HomotopyContinuation.System(grad)
println("The system is of degree: ", d - 1)
Real_sol_lstsq = HomotopyContinuation.solve(sys)
real_pts_lege = HomotopyContinuation.real_solutions(Real_sol_lstsq; only_real=true, multiple_results=false);

The system is of degree: 3


In [13]:
function process_critical_points(real_pts, f, scale_factor)
    # Filter points within the domain [-1,1] × [-1,1]
    condition(point) = -1 < point[1] < 1 && -1 < point[2] < 1
    filtered_points = filter(condition, real_pts)

    # Extract coordinates
    h_x = Float64[point[1] for point in filtered_points]
    h_y = Float64[point[2] for point in filtered_points]

    # Scale points and evaluate function
    h_z = map(p -> f([p[1], p[2]]), zip(scale_factor * h_x, scale_factor * h_y))

    # Create DataFrame
    DataFrame(
        x=scale_factor * h_x,
        y=scale_factor * h_y,
        z=h_z
    )
end

# Usage example:
df_cheb = process_critical_points(real_pts_cheb, f, scale_factor)
df_lege = process_critical_points(real_pts_lege, f, scale_factor)

Row,x,y,z
,Float64,Float64,Float64
1,0.646182,-0.63055,0.967104
2,-0.722991,0.679183,8.24656e-11
3,0.0842985,0.26326,0.017505
4,-0.735435,-0.598579,-0.323934
5,0.027884,0.794077,0.126365
6,0.320913,0.909743,0.126748
7,-0.766735,-0.13545,-0.000666275


In [15]:
pol_cheb

ApproxPoly{Float64}([0.08389253573359783, 0.15574055757948674, -0.019266075254266546, -0.09449376277730333, 0.034987429296810736, 0.011332417231878451, -0.09952360781832402, -0.027526156604580188, 0.08982996217966067, -0.04124184568825464, 0.03540554388765322, 0.07111497921331629, 0.12383179180226041, 0.18001113848602915, -0.11741900307233277], 4, 0.030777449424258453, 10, 1.0, [0.9898214418809327 0.9898214418809327; 0.9096319953545184 0.9898214418809327; … ; -0.9096319953545182 -0.9898214418809327; -0.9898214418809327 -0.9898214418809327], [0.035602495969302365, 0.07220244599168561, 0.12258095841627496, 0.047444258543253334, 0.03793574100902086, 0.007118387632417144, 3.156291436111317e-5, 7.866410427528702e-9, 7.118429007715401e-13, 2.373467728832574e-16  …  0.029706039055192035, 0.0395079019070517, 0.03095820771666844, 0.013168606453964832, -0.00025779265657878067, -0.10877138205058838, -0.28600042524435726, -0.10520863733709486, -0.01904363640890293, -0.005627788850334865])

In [16]:
using GLMakie

# Extract coordinates and function values
coords = pol_cheb.scale_factor * pol_cheb.grid
z_coords = pol_cheb.z

if size(coords)[2] == 2  # Plot the 3D scatter plot if the dimensions are 2
    fig = Figure(size=(800, 600))
    ax = Axis3(fig[1, 1], title="Mixture of Gaussians Sample Points", xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

    # Plot the sampled data with a darker core
    scatter!(ax, coords[:, 1], coords[:, 2], z_coords, markersize=2, color=:black, label="Sampled Data Core")

    # Plot the sampled data with adjusted size and color to create a halo effect
    scatter!(ax, coords[:, 1], coords[:, 2], z_coords, markersize=4, color=z_coords, colormap=:viridis, label="Sampled Data Halo")

    # Plot the exact approximant critical points with adjusted size
    scatter!(ax, df_cheb.x, df_cheb.y, df_cheb.z, markersize=10, color=:orange, label="Chebyshev approximant critical points")

    scatter!(ax, df_lege.x, df_lege.y, df_lege.z, markersize=10, color=:yellow, label="Legendre approximant critical points")

    display(fig)
end

2024-12-02 17:36:55.221 julia[93143:993854] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-02 17:36:55.221 julia[93143:993854] +[IMKInputSession subclass]: chose IMKInputSession_Modern


GLMakie.Screen(...)

In [11]:
# Extract coordinates and function values
using Statistics: mean

if size(coords)[2] == 2
    fig = Figure(size=(800, 600))
    ax = Axis3(fig[1, 1], title="Mixture of Gaussians Surface Plot", xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

    # Create a regular grid for surface interpolation
    x_unique = unique(coords[:, 1])
    y_unique = unique(coords[:, 2])
    x_grid = range(minimum(x_unique), maximum(x_unique), length=100)
    y_grid = range(minimum(y_unique), maximum(y_unique), length=100)

    # Interpolate scattered data to regular grid
    z_surface = [mean(z_coords[findall(x -> isapprox(x[1], xi) && isapprox(x[2], yi),
        zip(coords[:, 1], coords[:, 2]))]) for xi in x_grid, yi in y_grid]

    # Plot surface
    surface!(ax, x_grid, y_grid, z_surface, colormap=:viridis, transparency=true)

    # Add original scatter points
    scatter!(ax, coords[:, 1], coords[:, 2], z_coords, markersize=2, color=:black)

    # Add critical points
    scatter!(ax, df.x, df.y, df.z, markersize=10, color=:red)

    display(fig)
end

MethodError: MethodError: no method matching keys(::Base.Iterators.Zip{Tuple{Vector{Float64}, Vector{Float64}}})
The function `keys` exists, but no method is defined for this combination of argument types.

Closest candidates are:
  keys(!Matched::Base.TermInfo)
   @ Base terminfo.jl:232
  keys(!Matched::Pkg.Registry.RegistryInstance)
   @ Pkg ~/.julia/juliaup/julia-1.11.1+0.x64.apple.darwin14/share/julia/stdlib/v1.11/Pkg/src/Registry/registry_instance.jl:449
  keys(!Matched::Result)
   @ HomotopyContinuation ~/.julia/packages/HomotopyContinuation/kuurJ/src/result.jl:116
  ...


In [29]:
# savefig(plt, "../data/figures/random_gaussian_function_plot.html")

Should plot the polynomial approximant too.